In [1]:
import pandas as pd
import os
os.chdir('C:/Users/Usuario/Desktop/ASTYANAX_EYE/Superficie/2_ProteinOrtho/2_ProteinOrtho_IDs')

Al archivo obtenido de ProteinOrtho (Ast-Hum) se agregaron los ID's (a2, b2) del proteoma obtenido a traves del NCBI

In [2]:
ortho = pd.read_csv('Ortho_Ast_homo.csv') 
ortho.head()

,Ast,a,a2,Homo,b,b2,evalue_ab,bitscore_ab,evalue_ba,bitscore_ba
0,Astyanax_mexicanus,NP_001278181.1,BMP4,Homo_sapiens,NP_001193.2,BMP4,4.450000e-168,591.0,2.080000e-168,591.0
1,Astyanax_mexicanus,NP_001278181.1,BMP4,Homo_sapiens,NP_001334843.1,BMP4,4.450000e-168,591.0,2.080000e-168,591.0
2,Astyanax_mexicanus,NP_001278181.1,BMP4,Homo_sapiens,NP_001334845.1,BMP4,4.450000e-168,591.0,2.080000e-168,591.0
3,Astyanax_mexicanus,NP_001278181.1,BMP4,Homo_sapiens,NP_570911.2,BMP4,4.450000e-168,591.0,2.080000e-168,591.0
4,Astyanax_mexicanus,NP_001278181.1,BMP4,Homo_sapiens,NP_570912.2,BMP4,4.450000e-168,591.0,2.080000e-168,591.0


In [3]:
ortho.shape

(157518, 10)

In [4]:
nod= pd.read_csv('Homo_nodos.csv')
nod.head()

,Model_nodos,NCBI_Product_accesion
0,ACVRL1,NP_000011.2
1,ADRB2,NP_000015.2
2,ADRB3,NP_000016.1
3,ALDOB,NP_000026.2
4,APOC3,NP_000031.1


In [5]:
# Asegúrate de que no hay espacios extra o diferencias en los valores
nod['NCBI_Product_accesion'] = nod['NCBI_Product_accesion'].astype(str).str.strip()
ortho['b  '] = ortho['b  '].astype(str).str.strip()

# Filtrado: conservar solo las filas de 'ortho' que tengan coincidencias en 'nod'
filtrado = ortho[ortho['b  '].isin(nod['NCBI_Product_accesion'])]

# Mostrar los primeros resultados
filtrado.head()

,Ast,a,a2,Homo,b,b2,evalue_ab,bitscore_ab,evalue_ba,bitscore_ba
5,Astyanax_mexicanus,NP_001278181.1,BMP4,Homo_sapiens,NP_001334841.1,BMP4,2.980000e-160,565.0,1.550000e-160,565.0
14,Astyanax_mexicanus,NP_001278189.1,CART2,Homo_sapiens,NP_004282.1,CARTPT,1.940000e-27,122.0,9.000000e-28,122.0
21,Astyanax_mexicanus,NP_001278191.1,CRY1A,Homo_sapiens,NP_001400389.1,CRY1,2.040000e-293,1008.0,9.000000e-294,1008.0
23,Astyanax_mexicanus,NP_001278192.1,FGF8A,Homo_sapiens,NP_006110.1,FGF8,5.640000e-85,314.0,2.660000e-85,314.0
25,Astyanax_mexicanus,NP_001278193.1,VSX2,Homo_sapiens,NP_878314.1,VSX2,6.720000e-108,391.0,2.950000e-108,391.0


In [6]:
filtrado.shape

(3689, 10)

In [7]:
#Unión de nodos del modelo con ortólogos
merged = pd.merge(nod, filtrado, left_on='NCBI_Product_accesion', right_on='b  ', how='left')

In [8]:
merged.head()

,Model_nodos,NCBI_Product_accesion,Ast,a,a2,Homo,b,b2,evalue_ab,bitscore_ab,evalue_ba,bitscore_ba
0,ACVRL1,NP_000011.2,Astyanax_mexicanus,XP_007246816.1,ACVRL1,Homo_sapiens,NP_000011.2,ACVRL1,3.080000e-153,542.0,1.440000e-153,542.0
1,ACVRL1,NP_000011.2,Astyanax_mexicanus,XP_015460684.1,ACVRL1,Homo_sapiens,NP_000011.2,ACVRL1,3.080000e-153,542.0,1.440000e-153,542.0
2,ACVRL1,NP_000011.2,Astyanax_mexicanus,XP_015460685.1,ACVRL1,Homo_sapiens,NP_000011.2,ACVRL1,3.080000e-153,542.0,1.440000e-153,542.0
3,ACVRL1,NP_000011.2,Astyanax_mexicanus,XP_022532622.1,ACVRL1,Homo_sapiens,NP_000011.2,ACVRL1,1.540000e-153,543.0,7.210000e-154,543.0
4,ACVRL1,NP_000011.2,Astyanax_mexicanus,XP_049318709.1,ACVRL1,Homo_sapiens,NP_000011.2,ACVRL1,3.080000e-153,542.0,1.440000e-153,542.0


In [9]:
# 4. Agrupar por nodo y accession ID — mantener lista de tuplas (ID, símbolo)
grouped = merged.groupby(['Model_nodos', 'b  ']).apply(
    lambda df: list(zip(df['a'], df['a2']))
).reset_index(name='Astyanax_ortologos')

# 5. Añadir símbolo del gen de los IDs de referencia
symbol_map = filtrado[['b  ', 'b2']].drop_duplicates().set_index('b  ')['b2']
grouped['Symbol'] = grouped['b  '].map(symbol_map)

# 6. Renombrar columna 'a' a 'NCBI_Product_accesion' para mantener consistencia
grouped = grouped.rename(columns={'b  ': 'NCBI_Product_accesion'})

# 7. Reordenar columnas
grouped = grouped[['Model_nodos', 'NCBI_Product_accesion', 'Astyanax_ortologos']]


C:\Users\Usuario\AppData\Local\Temp\ipykernel_7200\4189615555.py:2: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  grouped = merged.groupby(['Model_nodos', 'b  ']).apply(


In [10]:
grouped.head()

,Model_nodos,NCBI_Product_accesion,Astyanax_ortologos
0,A2M,NP_001334352.2,"[(XP_007236950.3, LOC103029544), (XP_022524556..."
1,AATF,NP_036270.1,"[(XP_007245269.3, AATF)]"
2,ABCA1,NP_005493.2,"[(XP_022522959.2, ABCA1A), (XP_022522960.2, AB..."
3,ABCA3,NP_001080.2,"[(XP_022520829.2, ABCA3B)]"
4,ABCB1,NP_001335874.1,"[(XP_049320497.1, ABCB4)]"


In [11]:
grouped.to_csv('Homo_Dic_Orto.csv')

In [12]:
merged.to_csv('Homo_Dic_merged.csv')